In [13]:
# Emma Gabriela Alfaro 
# A01740229

# Model design
from secrets import choice
import agentpy as ap
import random

moves = [(1,0),(0,1),(-1,0),(0,-1),(1,1),(1,-1),(-1,1),]

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

class DirtyCells(ap.Model):

    def setup(self):
        # Create agents (celdas sucias, robot)
        self.robot = ap.AgentList(self,5)
        self.trash = ap.AgentList(self,10)
       

        # Create grid (cuarto)
        self.room = ap.Grid(self, [self.p['M'], self.p['N']], track_empty=True)
        self.room.add_agents(self.robot,[(1,1)]*5)
        self.room.add_agents(self.trash,[(1,1)])     
        self.room.add_agents(self.trash, random=True)   

        # Initiate a dynamic variable for all dirty cells
        # Condition 0: dirty, 1: cleaning, 2: cleaned
        self.robot.type_agent = 0
        self.trash.type_agent = 1

    def step(self):

        robots = self.robot
        # Select cleaning cells


        # Start cleaning
        for robot in robots:
            for neighbor in self.room.neighbors(robot):
                if neighbor.type_agent == 1:
                    neighbor.type_agent = 2 # Neighbor starts being cleaned
                    break
            else:
                self.room.move_by(robot, random.choice(moves))
        
        trash_remaining = self.trash.select(self.trash.type_agent == 1)
        # Stop simulation if no dirty cell is left
        if len(trash_remaining) == 0:
            self.stop()

    def end(self):
        # Document a measure at the end of the simulation
        cleaned_cells = len(self.trash.select(self.trash.type_agent == 2))
        self.report('Percentage of cleaned cells',
                    cleaned_cells / len(self.trash))

# Define parameters

parameters = {
    'Dirty Cell Density': 0.6, # Percentage of grid covered by dirty cells
    'M': 10,
    'N': 10,
    'steps': 100,
}
# Create single-run animation with custom colors

def animation_plot(model, ax):
    room_size = parameters['M'] * parameters['N']
    initial_dirtyCells = parameters['Dirty Cell Density'] * room_size
    cleaned_cells = len(model.robot.select(model.robot.type_agent == 2))
    cleaning_percentage = float("{:.2f}".format( cleaned_cells / (initial_dirtyCells) * 100))


    attr_grid = model.room.attr_grid('type_agent')
    color_dict = {0:'#a9825a', 1:'#969696', 2:'#e5e5e5', None:'#ebdaca'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of dirty cells being cleaned\n"
                 f"Time-step: {model.t}, Dirty cells left: "
                 f"{(len((model.trash.select(model.trash.type_agent == 2)))/len(model.trash))*100}")


fig, ax = plt.subplots()
model = DirtyCells(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))